# Create cluster from a bulk

Create a spherical cluster from a slab.

<h2 style="color:green">Usage</h2>

1. Make sure to select Input Materials (in the outer runtime) before running the notebook.
1. Set notebook parameters in cell 1.1. below (or use the default values).
1. Click “Run” > “Run All” to run all cells. 
1. Scroll down to view results. 


## Notes

1. For more information, see [Introduction](Introduction.ipynb)
<!-- # TODO: use a hashtag-based anchor link to interface creation documention above -->


## 1. Prepare the Environment
### 1.1. Set up cluster parameters

In [ ]:
RADIUS = 4.5  # in Angstroms
VACUUM = 10.0  # in Angstroms on each side

### 1.2. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages

    await install_packages("", "../../config.yml")

### 1.3. Get input material

In [ ]:
from utils.jupyterlite import get_materials

materials = get_materials(globals())

### 1.4. Create a slab of sufficient size

In [ ]:
from mat3ra.made.tools.build.supercell import create_supercell
from mat3ra.made.tools.build.slab import create_slab, SlabConfiguration
from utils.visualize import visualize_materials as visualize

DEFAULT_SLAB_PARAMETERS = {
    "miller_indices": (0, 0, 1),
    "thickness": 1,
    "vacuum": 0.0,
    "use_orthogonal_z": True,
}

slab_config = SlabConfiguration(
    bulk=materials[0],
    miller_indices=DEFAULT_SLAB_PARAMETERS["miller_indices"],
    thickness=DEFAULT_SLAB_PARAMETERS["thickness"],
    vacuum=DEFAULT_SLAB_PARAMETERS["vacuum"],
    use_orthogonal_z=DEFAULT_SLAB_PARAMETERS["use_orthogonal_z"],
)

slab = create_slab(slab_config)
length = slab.lattice.a
sufficient_n = int(2 * RADIUS / length) + 1

slab = create_supercell(slab, scaling_factor=[sufficient_n, sufficient_n, sufficient_n])

visualize([{"material": slab, "title": "Original material"}])
visualize([{"material": slab, "title": "Original material"}], rotation="-90x")

## 2. Create the Target Material
### 2.1. Create spherical cluster with vacuum


In [ ]:
from mat3ra.made.tools.modify import filter_by_sphere, add_vacuum, add_vacuum_sides

cluster = filter_by_sphere(material=slab, center_coordinate=[0.5, 0.5, 0.5], radius=RADIUS)
cluster = add_vacuum(cluster, VACUUM, to_bottom=True, on_top=True)
cluster = add_vacuum_sides(cluster, VACUUM, on_x=True, on_y=True)

## 3. Visualize the Result(s)

In [ ]:
visualize([{"material": slab, "title": "Original material"},
           {"material": cluster, "title": f"Cluster"}])

visualize([{"material": slab, "title": "Original material"},
           {"material": cluster, "title": f"Cluster"}], rotation="-90x")

## 4. Pass data to the outside runtime

In [ ]:
from utils.jupyterlite import set_materials

cluster.name = f"{materials[0].name} Cluster R={RADIUS}A"
set_materials(cluster)